# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686554


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:19,  2.83s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:54,  2.04s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:38,  1.48s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:27,  1.08s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  5.90it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.94it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  5.19it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  5.01it/s]

BridieK                               0.000627
not-logged-in-c3ca171c6551a9bcc607    0.000509
Linda_J._Berkel                       0.000450
shirogane                             0.000498
asucich                               0.000928
jarmo                                 0.024254
not-logged-in-f7e2260ed4a9517a3e4a    0.051681
djswanso                              0.000622
thedarktree                           0.006151
ElisabethB                            0.035983
ComeMyChild                           0.001317
not-logged-in-6ead6661c7dcef165774    0.006686
not-logged-in-6e654a118e00ea29ed67    0.011383
not-logged-in-a76dd994176a1ff14298    0.001023
not-logged-in-5f30b2bb9b7f9d49da41    0.005781
peakscience7                          0.000786
mhshakouri                            0.072531
pangeli5                              0.000523
not-logged-in-6fef3545124f3817aa11    0.003947
rhondacard                            0.005198
Lavadude                              0.000851
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())